***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [23]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-03eb  GCE       4                                             RUNNING  us-central1-a


# Imports & Setup

In [24]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [25]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import math
import builtins as pybuiltins


import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  2 07:23 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [27]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [28]:
spark

In [29]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'noam209263805' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and not b.name.startswith("postings_gcp"):
        paths.append(full_path+b.name)

In [30]:
parquetFile = spark.read.parquet(*paths)
# doc_title_pairs = parquetFile.select("title", "id").rdd
# doc_anchor_pairs = parquetFile.select("anchor_text", "id").rdd
doc_body_pairs = parquetFile.select("text", "id").rdd

In [31]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [32]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

24/03/02 08:39:42 WARN SparkContext: The path /home/dataproc/inverted_index_gcp.py has been added already. Overwriting of added paths is not supported in the current version.


In [33]:
from inverted_index_gcp import InvertedIndex

**Project**


In [34]:
# stop words
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)


# tokenize and stemming
def tokenize_stemming(text):
    list_token = []
    stemmer = PorterStemmer()
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    for token in tokens:
        if token not in english_stopwords:
            list_token.append(stemmer.stem(token))
    return list_token

# ---------------------new------------------------------
def tokeniz_clean(text):
  list_token = []
  RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
  stemmer = PorterStemmer()
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  for token in tokens:
    if token not in english_stopwords: 
      list_token.append(stemmer.stem(token))
  return list_token


In [35]:
# tf
# normalizion by max frequency term
def term_frequency(text, id):
#     tokens_stem = tokenize_stemming(text)
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]

    # Count the frequency of each word that is not a stopword
    word_freq = {}
    doc_size = 0

    for token in tokens:
        doc_size += 1
        if token not in all_stopwords:
            if token not in word_freq:
                word_freq[token] = 1
            else:
                word_freq[token] += 1
    # freq after normalizied
    lst_tuples = [(token, (id, freq)) for token, freq in word_freq.items()]
    return lst_tuples


def reduce_word_counts(unsorted_pl):
  sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])
  return sorted_pl

# df
def calculate_df(postings):
  rdd_token_doc_freq = postings.map(lambda x: (x[0], len(x[1]))).reduceByKey(lambda a, b: a + b)
  return rdd_token_doc_freq


# doc len
def doc_lenght(text, doc_id):
  tokens =  tokeniz_clean(text)
  return((doc_id,len(tokens)))

# def len_doc(text):
#   count = 0
#   tokens =  tokeniz_clean(text)
#   for token in tokens:
#       count+=1
#   return count


In [36]:


NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings, index_name):
  def map_to_buckets(pair):
        word, posting_list = pair
        bucket_id = token2bucket_id(word)
        return bucket_id, [(word, posting_list)]

  def reduce_to_disk(bucket, word_posting_lists):
    # Write posting lists for the bucket to disk
    # posting_locs = InvertedIndex.write_a_posting_list((bucket, word_posting_lists), 'postings_gcp', bucket_name)
    return posting_locs

  # # Partition the postings into buckets
  # buckets = postings.map(map_to_buckets)

  # # Group postings by bucket ID
  # grouped_buckets = buckets.reduceByKey(lambda x, y: x + y)

  # # Write each bucket to disk and get posting locations
  # bucket_posting_locs = grouped_buckets.map(lambda x: reduce_to_disk(*x))
  # return bucket_posting_locs

  # ---------------------------------new ---------------------------
  tempRDD = postings.map(lambda x:(token2bucket_id(x[0]),[(x[0],x[1])]))
  ldRDD = tempRDD.reduceByKey(lambda a,b: a+b)
  posting_list =ldRDD.map(lambda x: InvertedIndex.write_a_posting_list(x, f'bucket_{index_name}', bucket_name))
  return posting_list
  # -----------------------------------------------------------------


def create_posting_locs(index_name, postings_filtered):
  posting_locs_list = partition_postings_and_write(postings_filtered, index_name).collect()

  super_posting_locs = defaultdict(list)
  for blob in client.list_blobs(bucket_name, prefix= f'bucket_{index_name}'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)
  return super_posting_locs

In [37]:
def create_index(index_name, doc_pairs):

    word_counts = doc_pairs.flatMap(lambda x: term_frequency(x[0], x[1]))
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    postings_filtered = postings.filter(lambda x: len(x[1])>50)

    dl_rdd = doc_pairs.map(lambda x: doc_lenght(x[0], x[1]))
#     dl = dl_rdd.collectAsMap()
    w2df = calculate_df(postings_filtered)
    w2df_dict = w2df.collectAsMap()

    # Create inverted index instance
    inverted = InvertedIndex()
    inverted.df = w2df_dict
    inverted.N = doc_pairs.count()
    inverted.doc_len = dict(dl_rdd.collect())
    inverted.posting_locs = create_posting_locs(index_name, postings_filtered)

    inverted.write_index('.', f'index_{index_name}')

    index_src = f"index_{index_name}.pkl"
    index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
    !gsutil cp $index_src $index_dst
    
#     return inverted

In [38]:
TUPLE_SIZE = 6
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing


def read_posting_list(inverted, w):
  with closing(MultiFileReader(".", bucket_name)) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [39]:
# weight = tf*idf
def calculate_tf_idf(index, term, tf, doc_id):
  tf_idf = tf* (math.log2(index.N/index.df[term]))
  return tf_idf


def cosine_similarity(query, index):
    """ Returns: {doc_id:cosine score} """

    dict_cosine_sim = {}
    doc_weights_dict = {}
    query_dict = dict(term_frequency(query ,0))

    # create lists of keys and values from the query_dict
    query_list_keys = list(query_dict.keys())
    query_list_values = list(query_dict.values())

    # create list of items from index.df dictionery keys
    index_df_list_keys = list(index.df.keys())


    for term in query_list_keys:
      w_term_query = (query_dict[term][1]/len(query))* math.log2(index.N/index.df[term])

      if term in index_df_list_keys:
        posting_list = index.read_a_posting_list(".", term, bucket_name)
        
        for doc_id, freq in posting_list:  
          w_term_doc = calculate_tf_idf(index, term, freq, doc_id)
          if doc_id in dict_cosine_sim.keys():
            dict_cosine_sim[doc_id] += (w_term_doc)*(w_term_query)
            doc_weights_dict[doc_id] += (w_term_doc) ** 2
          else:
            dict_cosine_sim[doc_id] = (w_term_doc)*(w_term_query)
            doc_weights_dict[doc_id] = (w_term_doc) ** 2


    for doc_id in list(dict_cosine_sim.keys()):
      Word_doc_id__weight = doc_weights_dict[doc_id];
      dict_cosine_sim[doc_id] /= ( math.sqrt(pybuiltins.sum(value[1] ** 2 for value in query_list_values) * Word_doc_id__weight ))



    sorted_docs = sorted(dict_cosine_sim.items(), key=lambda x: x[1], reverse=True)

    print(sorted_docs)
    return dict_cosine_sim


In [40]:
# def BM25(query, index, k, b, avg_doclen):
#     bm_score = {}
#     # create lists of keys and values from the query_dict
#     query_list_keys = list(query_dict.keys())
#     query_list_values = list(query_dict.values())
#     # create list of items from index.df dictionery keys
#     index_df_list_keys = list(index.df.keys())
    
#     idf_dict = calc_idf(query, N, index)
    
#     for term in query_list_keys:
#         if term in index_df_list_keys:
#             posting_list = index.read_a_posting_list(".", term, bucket_name)
#             for doc_id, freq in posting_list:
#                 B = 1 - b + b * (index.dl[doc_id] / avg_doclen)
#                 idf = idf_dict[term]
#                 tf = freq / index.dl[doc_id]
#                 if doc_id in bm_score:
#                     bm_score[doc_id] += (idf * freq * (k + 1)) / ((freq + B * k))
#                 else:
#                     bm_score[doc_id] = (idf * freq * (k + 1)) / ((freq + B * k))
    
#     return dict(sorted(bm_score.items(), key=lambda x: x[1], reverse=True))
            

In [41]:
import time
def search_by_title(inverted, query):
    """ Returns: n best docs """
    start_time = time.time()  # Record the start time

    query_filterd = tokenize_stemming(query)
    cosSim_score = cosine_similarity(query, inverted)

    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Function execution time:", execution_time, "seconds")
    return cosSim_score

In [42]:
create_index("body", doc_body_pairs)
# idx = InvertedIndex.read_index("postings_gcp", 'index_title', bucket_name)


Copying file://index_body.pkl [Content-Type=application/octet-stream]...
- [1 files][ 62.9 MiB/ 62.9 MiB]                                                
Operation completed over 1 objects/62.9 MiB.                                     


In [22]:
idx = InvertedIndex.read_index("postings_gcp", 'index_body', bucket_name)
# idx.read_a_posting_list(".", "genetic", bucket_name)
# idx.doc_len

{4045403: 4,
 4045413: 2,
 4045419: 2,
 4045426: 2,
 4045432: 1,
 4045456: 3,
 4045466: 2,
 4045471: 2,
 4045479: 2,
 4045516: 2,
 4045519: 2,
 4045523: 1,
 4045525: 5,
 4045532: 1,
 4045544: 2,
 4045546: 1,
 4045554: 4,
 4045575: 2,
 4045577: 2,
 4045581: 4,
 4045584: 2,
 4045604: 2,
 4045607: 6,
 4045640: 5,
 4045651: 2,
 4045659: 5,
 4045661: 8,
 4045672: 4,
 4045691: 2,
 4045698: 2,
 4045705: 2,
 4045710: 2,
 4045717: 2,
 4045722: 2,
 4045725: 2,
 4045760: 1,
 4045770: 3,
 4045791: 2,
 4045792: 4,
 4045815: 8,
 4045829: 5,
 4045840: 2,
 4045849: 1,
 4045877: 1,
 4045883: 1,
 4045890: 1,
 4045901: 2,
 4045909: 2,
 4045920: 2,
 4045928: 4,
 4045953: 1,
 4045974: 1,
 4045978: 2,
 4045982: 1,
 4045991: 2,
 4046019: 2,
 4046036: 2,
 4046050: 4,
 4046061: 2,
 4046091: 2,
 4046117: 2,
 4046133: 3,
 4046154: 2,
 4046176: 2,
 4046178: 2,
 4046182: 4,
 4046185: 2,
 4046193: 3,
 4046194: 4,
 4046212: 2,
 4046218: 1,
 4046265: 2,
 4046272: 5,
 4046280: 2,
 4046286: 3,
 4046303: 1,
 4046316: 3,

In [ ]:
# query = "genetic"
# res = search_by_title(idx, query)

In [53]:
# create_index("body", doc_body_pairs)
idx = InvertedIndex.read_index("postings_gcp", 'index_body', bucket_name)

In [54]:
idx.N

6348910

In [58]:
query = "hello"
# res = search_by_title(idx, query)
# cosine_similarity(query, idx)
posting_list = idx.read_a_posting_list(".", query, bucket_name)
print(posting_list)

[(316, 65536, 0), (56098819, 880, 880), (65536, 74711041, 1), (1242, 458752, 0), (87490563, 1446, 1446), (65536, 95092740, 4), (1453, 655360, 0), (95420419, 1828, 1828), (65536, 129040385, 1), (2100, 262144, 0), (139329537, 2162, 2162), (131072, 145293313, 1), (2398, 65536, 0), (191561730, 3374, 3374), (65536, 227016705, 1), (3735, 589824, 0), (252051457, 4015, 4015), (524288, 265486338, 2), (4052, 65536, 0), (267780100, 4110, 4110), (131072, 283508737, 1), (4840, 131072, 0), (348061697, 5321, 5321), (65536, 356057089, 1), (5551, 65536, 0), (378994689, 6021, 6021), (524288, 397672458, 10), (6340, 65536, 0), (445579272, 7218, 7218), (65536, 492568577, 1), (8222, 65536, 0), (567148549, 8691, 8691), (65536, 571211779, 3), (9048, 65536, 0), (599064578, 9646, 9646), (131072, 632225794, 2), (9838, 393216, 0), (645201924, 10078, 10078), (65536, 682164226, 2), (10482, 65536, 0), (687079425, 10707, 10707), (65536, 721682440, 8), (11168, 131072, 0), (732561415, 11530, 11530), (65536, 766836737, 